In [1]:
###########################################################
# Célula 1
# Criação das bases em CSV, para facilitar meus estudos equnato estiver offline
#############################################################

#Importação dos módulos desta célula de código
from matminer.datasets.convenience_loaders import load_elastic_tensor
from matminer.featurizers.structure import CoulombMatrix
from matminer.featurizers.conversions import StrToComposition
from matminer.featurizers.composition import ElementProperty
from datetime import datetime
import pandas as pd
import logging

print("Start!")

#marca inicio dete bloco
t1 = datetime.now()

#Criação e configuração do log
log = logging.getLogger('Ex2-2')
log.setLevel(logging.DEBUG)
fh = logging.FileHandler('/home/luis/Downloads/Ex2-2.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s : %(message)s'))
log.addHandler(fh)

#####################################################################
# Criação dos dados de trino e teste com a base Elastic Tensor
#####################################################################
log.info("Start!")
#obter os dados do elastic tensor e remover colunas não utilizadas
df = load_elastic_tensor()  # loads dataset in a pandas DataFrame object
log.info("Terminada leitura do Elastic Tensor.")
#removendo linhas com mais de 30 átomos (Nsites > 30)
df = df[df.nsites<=30]
log.info("Removidas as linhas com nsites > 30.")
df.to_csv("/home/luis/Downloads/elastic_tensor.csv", index=False)
unwanted_columns = ["volume", "nsites", "compliance_tensor", "elastic_tensor", 
                    "elastic_tensor_original", "K_Voigt", "G_Voigt", "K_Reuss", "G_Reuss", 
                    "G_VRH", 'elastic_anisotropy']
df = df.drop(unwanted_columns, axis=1)
log.info("Removidas colunas que não serão utilizadas.")

#Criação do ndarray "Y"
y_K_VRH = df['K_VRH'].values
log.info("Criado o ndarray com os dados para 'Y'")
#Exporta em CSV para facilitar o trabalho quando eu estiver offline.
pd.DataFrame(y_K_VRH).to_csv("/home/luis/Downloads/y_K_VRH.csv", index=False) 
log.info("Criado o arquivo y_K_VRH.CSV com os dados para 'Y' para importar nos módulos posteriores.")
log.info("Ínicio dos dados de y_K_VRH:")
log.info(pd.DataFrame(y_K_VRH).head())

###################################################
# Primeiro featurizer: CoulombMatrix
###################################################
#Criação do dataframe "X" com a coulombmatrix
X = df['structure'].as_matrix()
log.info("Criados dados de X como matriz para usar o CoulombMatrix como Flatten")
cm = CoulombMatrix(flatten=True).fit(X)
log.info("Criada a estrutura da CoulombMatrix com dados de X")
X_CM = cm.featurize_dataframe(df.copy(), "structure")
log.info("Criado um DataFrame X_CM, cópia de X, incluindo os dados da CoulombMatrix ")
# exclui demais colunas, para ficar somente os dados da coulombmatrix
excluded = ["K_VRH", "formula", "material_id", 
           "poisson_ratio", "structure", "space_group"]
X_CM = X_CM.drop(excluded, axis=1)
log.info("Excluídas de X_CM as colunas que não serão usadas.")
#Exporta em CSV para facilitar o trabalho quando eu estiver offline.
X_CM.to_csv("/home/luis/Downloads/X_CM.csv", index=False) 
log.info("Criado o arquivo X_CM.CSV com os dados de X_CM para importar nos módulos posteriores.")
log.info("Início do dataframe com a CoulombMatrix:")
log.info(X_CM.head())

# Inclui couna "composition" para os próximos featurizers
df = StrToComposition().featurize_dataframe(df, "formula")
log.info("Incluido no Dataframe inicial os dados de Composition (feturizer), para utilização na criação do ElementProperty")

###################################################
# Segundo featurizer: ElementProperty
###################################################
#Criação do dataframe "X" com as informações de EwaldEnergy
ep = ElementProperty.from_preset(preset_name="magpie")
log.info("Criado elemento para featurizer ElementProperty com as informações de EwaldEnergy")
X_EP = ep.featurize_dataframe(df.copy(), col_id="composition")  # Inlui a coluna "composition"
log.info("Criado um DataFrame X_EP, cópia de X, incluindo os dados de ElementProperty")
#exclui demais colunas, para ficar somente os dados de ElementProperty
excluded = ["K_VRH", "formula", "material_id", 
            "poisson_ratio", "structure", "space_group", "composition"]
X_EP = X_EP.drop(excluded, axis=1)
log.info("Excluídas de X_EP as colunas que não serão usadas.")
#Exporta em CSV para facilitar o trabalho quando eu estiver offline.
X_EP.to_csv("/home/luis/Downloads/X_EP.csv", index=False) 
log.info("Criado o arquivo X_EP.CSV com os dados de X_EP para importar nos módulos posteriores.")
log.info("Início do dataframe com o ElementProperty")
log.info(X_EP.head())

#maca fim do bloco e tempo gasto
log.info("Finish!")
log.info("Tempo gasto %s" % (datetime.now() - t1))

print("Finish!")






Start!


/home/luis/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:58: FutureWarning:

Method .as_matrix will be removed in a future version. Use .values instead.



/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part

/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part

/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part

/home/luis/anaconda3/lib/python3.6/site-packages/matminer/featurizers/structure.py:618: ComplexWarning:

Casting complex values to real discards the imaginary part




Finish!


In [4]:
###########################################################
# Célula 2
# Predição com estimator KernelRidge e CoulombMatrix 
#############################################################

# evitando as mensagens dw warning
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
 

#Importação dos módulos desta célula de código
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from matplotlib import pyplot
from datetime import datetime
import pandas as pd
import matplotlib.patches as mpatches
import numpy
import logging

print("Start!")

#marca inicio dete bloco
t1 = datetime.now()

#Criação e configuração do log
log = logging.getLogger('Ex2-2')
log.setLevel(logging.INFO) # ou logging.DEBUG
fh = logging.FileHandler('/home/luis/Downloads/Ex2-2-Celula2.log')
fh.setLevel(logging.INFO) # ou logging.DEBUG
fh.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s : %(message)s'))
log.addHandler(fh)


# importação dos CSVs para os Dataframes
log.info("Start!")
y_K_VRH = pd.read_csv("/home/luis/Downloads/y_K_VRH.csv")
log.info("Carregado o dataframe com y_K_VRH")
log.info(y_K_VRH.columns)
log.info(y_K_VRH.describe())
X_EP = pd.read_csv("/home/luis/Downloads/X_EP.csv")
log.info("Carregado o dataframe com X_EP")
log.info(X_EP.columns)
log.info(X_EP.describe())
X_CM = pd.read_csv("/home/luis/Downloads/X_CM.csv")
log.info("Carregado o dataframe com y_X_CM")
log.info(X_CM.columns)
log.info(X_CM.describe())


#Separação dos dados de treino e teste
train_X_CM, val_X_CM, train_y_CM, val_y_CM = train_test_split(X_CM, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com CoulombMatrix: X_CM")
log.info(train_X_CM.describe())
log.info(val_X_CM.describe())
train_X_EP, val_X_EP, train_y_EP, val_y_EP = train_test_split(X_EP, y_K_VRH, random_state=1)
log.info("Criados os dados de train e test para o Dtaframe com ElementProperty: X_EP")
log.info(train_X_EP.describe())
log.info(val_X_EP.describe())

#criação de lista com dados de treino e teste (tentar tirar colchetes)
featurizers = [{"featurizer":"CoulombMatrix",
         "train_X": train_X_CM,
         "train_y": train_y_CM,
         "val_X": val_X_CM,
         "val_y": val_y_CM},
        {"featurizer":"ElementProperty",
         "train_X": train_X_EP,
         "train_y": train_y_EP,
         "val_X": val_X_EP,
         "val_y": val_y_EP}
       ]




# Lista de Kernels que serão utilizados e respectivas faixas de parâmetros para avaliação
# Os parâmetros utilizados são os disponíveis para os estimators. Este parâmentros podem ser 
# obtidos com a função get_params() de cada estimator
# Para os que vou sar neste estudo, são:
# KerlRidge -> 
# {alpha': 1, 'coef0': 1, 'degree': 3, 'gamma': None, 'kernel': 'linear', 'kernel_params': None}
# SVR ->
# {'C': 1.0, 'cache_size': 200, 'coef0': 0.0, 'degree': 3, 'epsilon': 0.1, 'gamma': 'auto_deprecated', 
# 'kernel': 'rbf', 'max_iter': -1, 'shrinking': True, 'tol': 0.001, 'verbose': False}
# Os valores iniciais de cada parâmetros foram definido em um range específico e depois 
# serão ajustados estes ranges conforme os resultados:
# alpha: Fibonacci de 1 a 5
# gamma: 0.1, 0.3, 0.5, 0.8
# degree: Fibonacci de 1 a 5
# coef0: 0.1, 0.3, 0.5, 0.8
param_KR = [{'kernel': ['linear'], 
                     'alpha': [1, 2, 3, 5]}, 
                    {'kernel': ['rbf'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8]},  
                     {'kernel': ['polynomial'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8],
                     'degree': [1, 2, 3, 5],
                     'coef0': [0.1, 0.3, 0.5, 0.8]},
                    {'kernel': ['cosine'], 
                     'alpha': [1, 2, 3, 5]}, 
                    {'kernel': ['laplacian'], 
                     'alpha': [1, 2, 3, 5], 
                     'gamma': [0.1, 0.3, 0.5, 0.8]}
                    ]
param_SVR = [{'kernel': ['linear'],  #o kernel linear só com um parametro demrou 20mn. para executar 
                     'C': [1]} #, 2, 3, 5]}, 
                   # {'kernel': ['rbf'], 
                   #  'C': [1, 2, 3, 5], 
                   #  'gamma': [0.1, 0.3, 0.5, 0.8]},  
                   #  {'kernel': ['polynomial'], 
                   #  'C': [1, 2, 3, 5], 
                   #  'gamma': [0.1, 0.3, 0.5, 0.8],
                   #  'degree': [1, 2, 3, 5],
                   #  'coef0': [0.1, 0.3, 0.5, 0.8]},
                   # {'kernel': ['cosine'], 
                   #  'C': [1, 2, 3, 5]}, 
                   # {'kernel': ['laplacian'], 
                   #  'C': [1, 2, 3, 5], 
                   #  'gamma': [0.1, 0.3, 0.5, 0.8]}
                    ]
estimators = [{"name": "KernelRidge",
               "estimator": KernelRidge(),
              "parameters": param_KR},
              {"name": "SVR",
              "estimator": SVR(),
             "parameters": param_SVR},
          ]
log.info("Criados os parâmetros para os estimators")
log.info(estimators)

# scores para avaliação de kernels lineares
# https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics.pairwise
scores = ['r2', 'neg_mean_absolute_error',  'neg_mean_squared_error']
log.info("Scores para o GridSearchCV:")
log.info(scores)

#laço para avaliação dos estimators
for estimator in estimators:
    #laço para avaliação dos diverso featurizers
    for featurizer in featurizers:
        # esta lista de scores pode ser passada direta para o parametro scoring do GridSearchCV, 
        #mas foi usada em um laço de "for" para melhor clareza no log de execução
        for score in scores: 
    
            print("-----------------------------------------------------------")
            print()
            print("# Avaliando: Estimator=%s, Featurizer=%s e Score = %s" % 
                  (estimator["name"], featurizer["featurizer"], score))
            print()
            log.info("# Avaliando: Estimator=%s, Featurizer=%s e Score = %s" % 
                  (estimator["name"], featurizer["featurizer"], score))
            
            clf = GridSearchCV(estimator["estimator"], estimator["parameters"], cv=2, verbose=5,  
                       scoring='%s' % score, n_jobs=-1)
            log.info("Criado o objeto GridSearchCV:")
            log.info(clf)

            print()
            print("Score com os dados de treino:")
            log.info("Score com os dados de treino:")

           
            clf.fit(featurizer["train_X"], featurizer["train_y"])
            log.info("Concluído o 'fit' com os dados de treino.")

            log.info("Melhores parâmetros encontrados:")
            log.info(clf.best_params_)
            print("Melhores parâmetros encontrados:")
            print(clf.best_params_)
    
            means = clf.cv_results_['mean_test_score']
            stds = clf.cv_results_['std_test_score']
            for mean, std, params in zip(means, stds, clf.cv_results_['params']):
                #print("%0.3f (+/-%0.03f) for %r"
                #      % (mean, std * 2, params))
                log.info("%0.3f (+/-%0.03f) for %r"
                      % (mean, std * 2, params))

            print()
    
log.info("Finish!")
print("Finish!")


Start!
-----------------------------------------------------------

# Avaliando: Estimator=SVR, Featurizer=CoulombMatrix e Score = r2


Score com os dados de treino:
Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   8 | elapsed: 13.2min remaining: 22.0min
[Parallel(n_jobs=-1)]: Done   5 out of   8 | elapsed: 13.2min remaining:  7.9min


KeyboardInterrupt: 